In [ ]:
  !git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 27.84 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data= {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key =  row[2]
    key = key.lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]


In [ ]:
emotion_list = list(data.keys())


In [ ]:
import os
os.mkdir('master_data1')
os.mkdir('master_data1/training')
os.mkdir('master_data1/testing')


In [ ]:
for emotion in emotion_list:
  os.mkdir('master_data1/training/'+emotion)
  os.mkdir('master_data1/testing/'+emotion)

In [ ]:
from shutil import copyfile
split_size = 0.8
#80 % images for the training data

for emotion, images in data.items():
  train_size = int(split_size * len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('/content/master_data1/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('/content/master_data1/testing',emotion,image)
    copyfile(source,dest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = tf.keras.models.Sequential([
      Conv2D(16,(3,3),activation = 'relu',input_shape=(100,100,3)),
      MaxPooling2D(pool_size = (2,2)),
      Conv2D(32,(3,3),activation='relu'),
      MaxPooling2D(pool_size = (2,2)),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(pool_size = (2,2)),
      Flatten(),
      Dense(1024, activation = 'relu'),
      Dense(8,activation = 'softmax')
])
model.compile(optimizer = Adam(learning_rate = 0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)             

In [ ]:
train_dir = '/content/master_data1/training'
testing_dir = '/content/master_data1/testing'

train_data_gen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_data_gen.flow_from_directory(
    train_dir,target_size=(100,100), class_mode='categorical',
    batch_size = 128
)
test_data_gen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_data_gen.flow_from_directory(
    testing_dir,target_size = (100,100), class_mode='categorical',
    batch_size = 128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor = 'val_acc',patience = 2, min_delta = 0.01)


In [ ]:
model.fit_generator(train_generator,epochs = 20, verbose =1, validation_data=test_generator,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
86/86 [==============================] - 19s 212ms/step - loss: 8.3371 - acc: 0.4341 - val_loss: 1.0575 - val_acc: 0.5011
Epoch 2/20
86/86 [==============================] - 18s 208ms/step - loss: 1.0344 - acc: 0.4901 - val_loss: 1.0459 - val_acc: 0.5011
Epoch 3/20
86/86 [==============================] - 18s 208ms/step - loss: 1.0427 - acc: 0.4972 - val_loss: 1.0431 - val_acc: 0.5011
